<a href="https://colab.research.google.com/github/uroojfatima18/AI-Agents/blob/main/context_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Task: Build a Dynamic Greeting System Using OpenAI SDK


Build a system using the OpenAI SDK where the agent can greet the user dynamically using their name and age.

The function should be integrated with the RunContextWrapper class, and the agent must be able to access the user's context without writing a new function each time.

The agent should automatically pull the user information (name and age) from the context object, making the system reusable and scalable.

In [ ]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.4/720.4 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from pydantic  import BaseModel
from agents import AsyncOpenAI,OpenAIChatCompletionsModel,RunConfig
from google.colab import userdata

gemini_api_key = userdata.get("GEMINI_API_KEY")

In [ ]:
# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

In [ ]:
from agents import set_default_openai_client, set_tracing_disabled
set_default_openai_client(external_client)
set_tracing_disabled(True)

In [28]:
import asyncio
from dataclasses import dataclass
from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class UserInfo():
  name:str
  age:int

@function_tool
async def  greetuser(context:RunContextWrapper[UserInfo]):
  name=context.context.name
  age=context.context.age
  return f"Hello {name}. A young {age} year's old girl .Welcome back!"



async def main():
  userinfo=UserInfo(name=" Urooj fatima" ,age ="19")

  agent = Agent[UserInfo](
        name="Assistant",
        tools=[greetuser],
        model=model,
        instructions=
        "Always use the greetuser tool to greet the user. "
        "Make sure to warmly welcome them to Panaversity using their age and name"
        )

  result = await Runner.run(
        starting_agent=agent,
        input="hello",
        context=userinfo,
    )
  print("Agent response:",result)

if __name__ == "__main__":
  asyncio.run(main())

Agent response: RunResult:
- Last agent: Agent(name="Assistant", ...)
- Final output (str):
    Hello Urooj fatima. A young 19 year's old girl .Welcome back! I hope you're ready to dive into some exciting learning experiences here at Panaversity!
- 3 new item(s)
- 2 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
